# Google Sheet Survey ETL Analysis: Pipeline Fundamentals

## Dylan K. Picart

This notebook walks through the **key logic** of the 10 main scripts in the ETL workflow, from extraction to Excel output.
Each section is a self-contained example of how the script works, and can be expanded for testing or demonstration.

---


## Extract

### 1. `extract/scrape_drive_links.py` (Google Drive Download)

> **Purpose:**
> Authenticates with Google Drive/Sheets API and scrapes all survey spreadsheets and tabs from a specified root folder.     
> Organizes found sheets/tabs by year and (anonymized) category (e.g., "Group A" / "Group B").      
> Saves a `links.yaml` config file mapping all available files/tabs for use by later scripts.   
> **Input:** Google Drive folder ID, Google API credentials     
> **Output:** YAML file (`data/configs/links.yaml`) with all survey sheet links/tabs, categorized for downstream ETL.

```python
import gspread
from google.oauth2.service_account import Credentials

creds = Credentials.from_service_account_file('secrets/service_account.json', scopes=[
    'https://www.googleapis.com/auth/drive.readonly'
])
gc = gspread.authorize(creds)

# Download all sheets from a folder (Google Sheets API example)
folder_id = "<your-folder-id>"
from googleapiclient.discovery import build
service = build('drive', 'v3', credentials=creds)
files = service.files().list(q=f"'{folder_id}' in parents", fields="files(id, name)").execute()['files']

for file in files:
    # Export each Google Sheet as CSV
    request = service.files().export_media(fileId=file['id'], mimeType='text/csv')
    with open(f"raw/{file['name']}.csv", "wb") as f:
        f.write(request.execute())
```

### 2. `extract/load_feedback_data.py`

> **Purpose:**
> Loads and merges survey data from cloud links (YAML), cleans headers/whitespace, merges across schools/tabs, and writes a raw, cleaned CSV for each group and year.   
> **Input:** `links.yaml`     
> **Output:** Cleaned CSV files (e.g., `data/raw/group_a_2021.csv`).

```python
import pandas as pd

df = pd.read_csv("raw/2021_GROUP_A.csv")
df.columns = [col.strip().replace("\xa0", " ") for col in df.columns]
df = df.dropna(how='all')
df.to_csv("processed/group_a/feedback_2021.csv", index=False)
```

---

### 3. `extract/raw_audit.py`

> **Purpose:**
> Extracts all question columns from the raw CSVs, catalogs unique questions, and exports a per-group audit CSV for mapping (e.g., `audit_group_a.csv`).       
> **Input:** Raw cleaned CSV    
> **Output:** Audit CSV with all question columns.

```python
import pandas as pd

df = pd.read_csv("processed/group_a/feedback_2021.csv")
questions = pd.DataFrame({"Raw Question": df.columns})
questions.to_csv("audit_group_a.csv", index=False)
```
---

## Transform

### 4. `transform/translate_spanish_csv.py`

> **Purpose:**
> Finds any non-English survey columns, translates headers and responses to English, and writes a new CSV with only English columns.    
> **Input:** Cleaned CSVs   
> **Output:** Translated CSVs (English only, e.g., `processed/group_a/feedback_2021_en.csv`).

```python
import pandas as pd
from googletrans import Translator

translator = Translator()
df = pd.read_csv("processed/group_a/feedback_2021_spanish.csv")
for col in df.columns:
    df[col] = df[col].astype(str).apply(lambda x: translator.translate(x, src='es', dest='en').text)
df.to_csv("processed/group_a/feedback_2021.csv", index=False)
```

---

### 5. `transform/audit_map.py`

> **Purpose:**
> Maps raw audit questions to canonical (standardized) question names using a mapping dictionary (e.g., `QCON_MAP`).
> Cleans columns, drops junk, and saves a normalized audit file for each group.     
> **Input:** Audit CSV  
> **Output:** Canonicalized audit file (e.g., `audit_group_a_canonical.csv`).

```python
import pandas as pd

audit = pd.read_csv("audit_group_a.csv")
QCON_MAP = {
    "How safe do you feel?": "Feelings of Safety",
    "Is there an adult you trust?": "Trusted Adult",
    # ... more sample mappings ...
}
audit["Canonical Question"] = audit["Raw Question"].map(QCON_MAP).fillna(audit["Raw Question"])
audit.to_csv("audit_group_a_canonical.csv", index=False)
```

---

### 6. `transform/summary_tables.py`

> **Purpose:**
> Generates summary tables (value counts, percentages, etc.) for every survey question/response, per group and year.    
> **Input:** Processed/cleaned CSV      
> **Output:** Summary CSVs (e.g., `processed/group_a/summary/group_a_2021_summary.csv`).

```python
import pandas as pd

df = pd.read_csv("processed/group_a/feedback_2021.csv")
summary = df.apply(pd.Series.value_counts).fillna(0).astype(int).T
summary.to_csv("processed/group_a/summary/2021_GROUP_A_summary.csv")
```

---

### 7. `transform/consolidate_responses.py`

> **Purpose:**
> Merges all per-question summary tables into a single master response table for each group.    
> **Input:** All summary tables for a group     
> **Output:** Master responses CSV (`processed/consolidated_responses_group_a.csv`).

```python
import pandas as pd
import glob

csvs = glob.glob("processed/group_a/summary/*.csv")
all_responses = pd.concat([pd.read_csv(f) for f in csvs], ignore_index=True)
all_responses.to_csv("processed/consolidated_responses_group_a.csv", index=False)
```

---

### 8. `transform/consolidate_questions.py`

> **Purpose:**
> Aggregates consolidated responses by canonical question, response type, and year,
> and appends a grouping column ("Overarching") for higher-level rollups.   
> **Input:** Master responses CSV   
> **Output:** Final consolidated questions file (`processed/consolidated_questions_group_a.csv`).

```python
import pandas as pd

df = pd.read_csv("processed/consolidated_responses_group_a.csv")
totals = df.groupby("Canonical Question").sum(numeric_only=True).reset_index()
totals.to_csv("processed/consolidated_questions_group_a.csv", index=False)
```

---

### 9. `transform/summarize_totals.py`

> **Purpose:**
> Merges and totals all canonical question files across groups and years,
> creating a master totals table for high-level analysis.   
> **Input:** All consolidated questions files   
> **Output:** `processed/canonical_question_totals.csv`.

```python
import pandas as pd

df_a = pd.read_csv("processed/consolidated_questions_group_a.csv")
df_b = pd.read_csv("processed/consolidated_questions_group_b.csv")
all_data = pd.concat([df_a, df_b], ignore_index=True)
all_data = all_data.groupby("Canonical Question").sum(numeric_only=True).reset_index()
all_data.to_csv("processed/canonical_question_totals.csv", index=False)
```

---

## Load

### 10. `load/load_to_excel.py`

> **Purpose:**
> Combines all master summary and response files into a single Excel workbook,
> with a "Master Summary" tab, group summary tabs, and per-year/group response tabs.    
> **Input:** All summary/response CSVs      
> **Output:** `processed/Master_Summary.xlsx` (anonymized name, all tabs generic, e.g., "Group A Summary", "2021 Group A Responses").

```python
import pandas as pd

with pd.ExcelWriter("processed/Survey_Master_Summary.xlsx", engine="xlsxwriter") as writer:
    pd.read_csv("processed/canonical_question_totals.csv").to_excel(writer, sheet_name="Master Summary", index=False)
    pd.read_csv("processed/consolidated_questions_group_a.csv").to_excel(writer, sheet_name="Group A Summary", index=False)
    pd.read_csv("processed/consolidated_questions_group_b.csv").to_excel(writer, sheet_name="Group B Summary", index=False)
```

---

### **Sample Survey Response Values for Testing (Example):**

| Respondent | Satisfaction | Trust |
| ---------- | ------------ | ----- |
| 1          | Satisfied    | Yes   |
| 2          | Unsatisfied  | No    |

*All code, files, and responses here are anonymized and safe for open-source/public sharing.*

### ETL structure

In [ ]:
scrape_drive_links.py
   ↓
load_feedback_data.py
   ↓
translate_spanish_csv.py
   ↓
raw_audit.py
   ↓
audit_map.py
   ↓
summary_tables.py
   ↓
consolidate_responses.py
   ↓
consolidate_questions.py
   ↓
summarize_totals.py
   ↓
load_to_excel.py

## Bash Command to run the ETL


#### `run_etl.sh`

```bash
#!/bin/bash
set -euo pipefail
# -e: Exit immediately if any command fails
# -u: Treat unset variables as errors
# -o pipefail: Fail if any command in a pipeline fails

# Function to run a Python script and check for errors
run_step() {
    script_path="$1"
    step_desc="$2"
    if [[ ! -f "$script_path" ]]; then
        echo "❌ ERROR: Script not found: $script_path"
        exit 1
    fi
    echo "=== $step_desc ==="
    if python "$script_path"; then
        echo "✅ SUCCESS: $step_desc"
    else
        echo "❌ ERROR: $step_desc failed."
        exit 1
    fi
}

# Function to check that a required file exists before continuing
check_exists() {
    path="$1"
    desc="$2"
    if [[ ! -e "$path" ]]; then
        echo "❌ ERROR: Expected output not found: $desc ($path)"
        exit 1
    fi
    echo "✅ Found: $desc ($path)"
}

echo "=== STARTING ETL PIPELINE ==="

# -------------------------------
# EXTRACT STAGE
# -------------------------------

run_step "scripts/extract/scrape_drive_links.py" "1. Scraping cloud drive links"
# Downloads all raw survey files (spreadsheets or CSVs) from cloud storage, and generates a mapping of available survey data.

run_step "scripts/extract/load_feedback_data.py" "2. Loading raw survey data"
# Loads and standardizes all survey files using the links mapping. Cleans headers, removes blanks, and organizes by group/year.

run_step "scripts/extract/translate_spanish_csv.py" "3. Translating non-English survey data"
# (If needed) Detects and translates survey files to English, producing new English-only CSVs.

run_step "scripts/extract/raw_audit.py" "4. Auditing survey question columns"
# Extracts all unique question columns for mapping and later canonicalization, outputs audit files for each group.

# -- Extract output checks --
check_exists "data/raw/group_a_2021.csv" "Group A raw data (2021)"
check_exists "data/raw/group_b_2021.csv" "Group B raw data (2021)"
check_exists "audit_group_a.csv" "Group A question audit"
check_exists "audit_group_b.csv" "Group B question audit"

# -------------------------------
# TRANSFORM STAGE
# -------------------------------

run_step "scripts/transform/audit_map.py" "5. Mapping audit questions to canonical names"
# Maps all question variants to a canonical list, cleans/join columns, and writes normalized audit files.

run_step "scripts/transform/summary_tables.py" "6. Generating summary tables"
# For each cleaned survey, generates response counts for every question, grouped by response type/scale.

run_step "scripts/transform/consolidate_responses.py" "7. Consolidating all response tables"
# Merges all per-question summary tables for each group into a single master response table for analysis.

run_step "scripts/transform/consolidate_questions.py" "8. Creating canonical question summaries"
# Aggregates response data by canonical question, year, and response type. Prepares for final reporting.

run_step "scripts/transform/summarize_totals.py" "9. Creating overall survey totals"
# Merges all group summaries into a unified master totals table for analysis and dashboarding.

# -- Transform output checks --
check_exists "data/processed/consolidated_questions_group_a.csv" "Group A consolidated questions"
check_exists "data/processed/consolidated_questions_group_b.csv" "Group B consolidated questions"
check_exists "data/processed/canonical_question_totals.csv" "Canonical question totals"

# -------------------------------
# LOAD STAGE (EXPORT/REPORTING)
# -------------------------------

run_step "scripts/load/load_to_excel.py" "10. Exporting all summaries to Excel"
# Combines all key summary and response files into a single Excel workbook, with clear tab naming for each group/year.

echo "=== PIPELINE COMPLETE ==="
```

---

### **Summary Table for Each Stage**

| Stage     | What It Does                                             | Example Output                                 |
| --------- | -------------------------------------------------------- | ---------------------------------------------- |
| Extract   | Downloads and standardizes all survey data               | `data/raw/group_a_2021.csv`                    |
| Transform | Maps, summarizes, and consolidates all questions/answers | `data/processed/canonical_question_totals.csv` |
| Load      | Combines all results into Excel for reporting            | `processed/Master_Summary.xlsx`                |